In [15]:
# 1. Preparation

## 1.1 Prepare for LLM

import os
import csv
from llama_index.core import Document, KnowledgeGraphIndex, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core.settings import Settings
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.llms.openai import OpenAI
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler
from llama_index.core import QueryBundle
from llama_index.core.schema import NodeWithScore
from llama_index.core.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever
from typing import List
from llama_index.core.query_engine import KnowledgeGraphQueryEngine
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine





In [16]:

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")

# Set up token counter
token_counter = TokenCountingHandler(
    tokenizer=None,  # Use default tokenizer
    verbose=True  # Set to False if you don't want per-query prints
)

callback_manager = CallbackManager([token_counter])

# Define LLM with the callback manager
llm = OpenAI(temperature=0, model="gpt-4o", callback_manager=callback_manager)

# Configure settings
Settings.llm = llm
Settings.chunk_size = 1024
Settings.callback_manager = callback_manager

## 1.2. Prepare for SimpleGraphStore as Graph Store

graph_store = SimpleGraphStore()


In [17]:
api_key = os.getenv("OPENAI_API_KEY")


In [18]:
# Function to print token usage and cost
def print_token_usage():
    print(f"Prompt Tokens: {token_counter.prompt_llm_token_count}")
    print(f"Completion Tokens: {token_counter.completion_llm_token_count}")
    print(f"Total Tokens: {token_counter.total_llm_token_count}")
    # Assuming a cost of $0.002 per 1K tokens for gpt-3.5-turbo-instruct
    cost = (token_counter.total_llm_token_count / 1000) * 0.002
    print(f"Estimated Cost: ${cost:.4f}")


In [19]:
# 2. Build or Load the Knowledge Graph and Vector Index

token_counter.reset_counts()

# Function to load documents
def load_documents():
    documents = []
    with open('/Users/akshit/Documents/Projects/Python-all/information-extraction/Guidelines/NCCN/half_ei_4_2024.csv', 'r') as f:
        csv_reader = csv.DictReader(f)
        print("CSV Column names:", csv_reader.fieldnames)
        for row in csv_reader:
            content = f"Node ID: {row['Node ID']}\n"
            content += f"Page Key: {row['Page Key']}\n"
            content += f"Page No: {row['Page No']}\n"
            content += f"Information: {row['Information']}\n"
            content += f"Footnotes: {row['Footnotes']}\n"
            documents.append(Document(text=content))
    return documents


In [20]:
# Check and load/create KnowledgeGraphIndex
if os.path.exists('./storage_graph'):
    print("Loading existing KnowledgeGraphIndex...")
    storage_context = StorageContext.from_defaults(persist_dir='./storage_graph', graph_store=graph_store)
    kg_index = load_index_from_storage(storage_context=storage_context)
else:
    print("Creating new KnowledgeGraphIndex...")
    documents = load_documents()
    storage_context = StorageContext.from_defaults(graph_store=graph_store)
    kg_index = KnowledgeGraphIndex.from_documents(
        documents,
        storage_context=storage_context,
        max_triplets_per_chunk=20,
        include_embeddings=True,
    )
    kg_index.storage_context.persist(persist_dir='./storage_graph')


Loading existing KnowledgeGraphIndex...


In [73]:
print(kg_index.docstore.)
# print(kg_index.docstore.docs['15b3c807-94d5-43f2-82d9-ff03ecc8dfb9'].relationships.keys())




Simple Document (Node) store.

    An in-memory store for Document and Node objects.

    Args:
        simple_kvstore (SimpleKVStore): simple key-value store
        namespace (str): namespace for the docstore

    


In [76]:
documents


list

In [21]:
# Check and load/create VectorStoreIndex
if os.path.exists('./storage_vector'):
    print("Loading existing VectorStoreIndex...")
    storage_context_vector = StorageContext.from_defaults(persist_dir='./storage_vector')
    vector_index = load_index_from_storage(storage_context=storage_context_vector)
else:
    print("Creating new VectorStoreIndex...")
    if 'documents' not in locals():
        documents = load_documents()
    vector_index = VectorStoreIndex.from_documents(documents)
    vector_index.storage_context.persist(persist_dir='./storage_vector')

print("\nToken usage after index creation/loading:")
print_token_usage()

Loading existing VectorStoreIndex...

Token usage after index creation/loading:
Prompt Tokens: 0
Completion Tokens: 0
Total Tokens: 0
Estimated Cost: $0.0000


In [16]:
# 5. Prepare for different query approaches

# 5.1 text-to-GraphQuery
# from llama_index.core.query_engine import KnowledgeGraphQueryEngine

# nl2kg_query_engine = KnowledgeGraphQueryEngine(
#     storage_context=kg_index.storage_context,
#     llm=llm,
#     verbose=True,
# )

/var/folders/bj/8gzt5tq51b1c9qlynhj982vc0000gn/T/ipykernel_69026/3198315662.py:6: DeprecationWarning: Call to deprecated class KnowledgeGraphQueryEngine. (KnowledgeGraphQueryEngine is deprecated. It is recommended to use the PropertyGraphIndex and associated retrievers instead.) -- Deprecated since version 0.10.53.
  nl2kg_query_engine = KnowledgeGraphQueryEngine(


NotImplementedError: SimpleGraphStore does not support get_schema

In [22]:


## 5.2 Graph RAG query engine
kg_rag_query_engine = kg_index.as_query_engine(
    include_text=False,
    retriever_mode="keyword",
    response_mode="tree_summarize",
)

## 5.3 Vector RAG query engine
vector_rag_query_engine = vector_index.as_query_engine()

In [23]:
## 5.4 Graph+Vector RAG query engine


class CustomRetriever(BaseRetriever):
    def __init__(self, vector_retriever, kg_retriever, mode="OR"):
        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        self._mode = mode


    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

vector_retriever = VectorIndexRetriever(index=vector_index)
kg_retriever = KGTableRetriever(index=kg_index, retriever_mode="keyword", include_text=False)
custom_retriever = CustomRetriever(vector_retriever, kg_retriever)


/var/folders/bj/8gzt5tq51b1c9qlynhj982vc0000gn/T/ipykernel_75148/3305116532.py:30: DeprecationWarning: Call to deprecated class KGTableRetriever. (KGTableRetriever is deprecated, it is recommended to use PropertyGraphIndex and associated retrievers instead.) -- Deprecated since version 0.10.53.
  kg_retriever = KGTableRetriever(index=kg_index, retriever_mode="keyword", include_text=False)


In [24]:
# from llama_index import get_response_synthesizer
# from llama_index.query_engine import RetrieverQueryEngine

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

graph_vector_rag_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

In [14]:
# 6. Query with all the Engines

query = "What are the initial steps for prostate cancer diagnosis?"

# token_counter.reset_counts()
# print("Text-to-GraphQuery result:")
# response_nl2kg = nl2kg_query_engine.query(query)
# print(response_nl2kg)
# print_token_usage()

token_counter.reset_counts()
print("\nGraph RAG result:")
response_graph_rag = kg_rag_query_engine.query(query)
print(response_graph_rag)
print_token_usage()

token_counter.reset_counts()
print("\nVector RAG result:")
response_vector_rag = vector_rag_query_engine.query(query)
print(response_vector_rag)
print_token_usage()

token_counter.reset_counts()
print("\nGraph + Vector RAG result:")
response_graph_vector_rag = graph_vector_rag_query_engine.query(query)
print(response_graph_vector_rag)
print_token_usage()

Prompt Tokens: 0
Completion Tokens: 0
Total Tokens: 0
Estimated Cost: $0.0000

Graph RAG result:
LLM Prompt Token Usage: 78
LLM Completion Token Usage: 31
LLM Prompt Token Usage: 129
LLM Completion Token Usage: 72
The initial steps for prostate cancer diagnosis typically include a digital rectal exam (DRE) and a prostate-specific antigen (PSA) blood test. These tests help in assessing the prostate gland and measuring PSA levels, which can indicate the presence of prostate cancer. If results from these tests are abnormal, further diagnostic procedures such as a prostate biopsy may be recommended.
Prompt Tokens: 207
Completion Tokens: 103
Total Tokens: 310
Estimated Cost: $0.0006

Vector RAG result:
Embedding Token Usage: 10
LLM Prompt Token Usage: 910
LLM Completion Token Usage: 124
The initial steps for prostate cancer diagnosis include performing a physical exam and a digital rectal exam (DRE) to confirm the clinical stage. It is also important to perform and/or collect prostate-speci

In [15]:
# 7. Conclusion

# Print total token usage and cost
print("\nTotal Token Usage:")
print_token_usage()

# Compare results and analyze performance


Total Token Usage:
Prompt Tokens: 997
Completion Tokens: 156
Total Tokens: 1153
Estimated Cost: $0.0023


In [18]:
# 6. Query with all the Engines

query = "At what times are we referencing the footnote a? a footnote is referenced when it is written in curly braces."

# token_counter.reset_counts()
# print("Text-to-GraphQuery result:")
# response_nl2kg = nl2kg_query_engine.query(query)
# print(response_nl2kg)
# print_token_usage()

token_counter.reset_counts()
print("\nGraph RAG result:")
response_graph_rag = kg_rag_query_engine.query(query)
print(response_graph_rag)
print_token_usage()

token_counter.reset_counts()
print("\nVector RAG result:")
response_vector_rag = vector_rag_query_engine.query(query)
print(response_vector_rag)
print_token_usage()

token_counter.reset_counts()
print("\nGraph + Vector RAG result:")
response_graph_vector_rag = graph_vector_rag_query_engine.query(query)
print(response_graph_vector_rag)
print_token_usage()


Graph RAG result:
LLM Prompt Token Usage: 92
LLM Completion Token Usage: 12
LLM Prompt Token Usage: 143
LLM Completion Token Usage: 19
There are no specific times mentioned for referencing the footnote "a" in the provided information.
Prompt Tokens: 235
Completion Tokens: 31
Total Tokens: 266
Estimated Cost: $0.0005

Vector RAG result:
Embedding Token Usage: 22
LLM Prompt Token Usage: 230
LLM Completion Token Usage: 15
The footnote "a" is not referenced in the provided context information.
Prompt Tokens: 230
Completion Tokens: 15
Total Tokens: 245
Estimated Cost: $0.0005

Graph + Vector RAG result:
Embedding Token Usage: 22
LLM Prompt Token Usage: 92
LLM Completion Token Usage: 12
LLM Prompt Token Usage: 239
LLM Completion Token Usage: 19
The information provided does not specify any times or instances when footnote "a" is referenced.
Prompt Tokens: 331
Completion Tokens: 31
Total Tokens: 362
Estimated Cost: $0.0007


In [19]:
# 6. Query with all the Engines

query = "when do we Perform imaging for staging?"

# token_counter.reset_counts()
# print("Text-to-GraphQuery result:")
# response_nl2kg = nl2kg_query_engine.query(query)
# print(response_nl2kg)
# print_token_usage()

token_counter.reset_counts()
print("\nGraph RAG result:")
response_graph_rag = kg_rag_query_engine.query(query)
print(response_graph_rag)
print_token_usage()

token_counter.reset_counts()
print("\nVector RAG result:")
response_vector_rag = vector_rag_query_engine.query(query)
print(response_vector_rag)
print_token_usage()

token_counter.reset_counts()
print("\nGraph + Vector RAG result:")
response_graph_vector_rag = graph_vector_rag_query_engine.query(query)
print(response_graph_vector_rag)
print_token_usage()


Graph RAG result:
LLM Prompt Token Usage: 76
LLM Completion Token Usage: 8
LLM Prompt Token Usage: 127
LLM Completion Token Usage: 15
The context does not provide specific information about when to perform imaging for staging.
Prompt Tokens: 203
Completion Tokens: 23
Total Tokens: 226
Estimated Cost: $0.0005

Vector RAG result:
Embedding Token Usage: 8
LLM Prompt Token Usage: 982
LLM Completion Token Usage: 32
Imaging for staging is performed during the workup for an initial prostate cancer diagnosis, specifically for regional prostate cancer (Any T, N1, M0).
Prompt Tokens: 982
Completion Tokens: 32
Total Tokens: 1014
Estimated Cost: $0.0020

Graph + Vector RAG result:
Embedding Token Usage: 8
LLM Prompt Token Usage: 76
LLM Completion Token Usage: 8
LLM Prompt Token Usage: 991
LLM Completion Token Usage: 23
Imaging for staging is performed during the workup for an initial prostate cancer diagnosis, particularly for regional prostate cancer.
Prompt Tokens: 1067
Completion Tokens: 31
To

In [20]:
# 6. Query with all the Engines

query = "How does the treatment for metastatic prostate cancer (M1) differ in its use of androgen deprivation therapy (ADT) compared to castration-sensitive prostate cancer (CSPC)?"

# token_counter.reset_counts()
# print("Text-to-GraphQuery result:")
# response_nl2kg = nl2kg_query_engine.query(query)
# print(response_nl2kg)
# print_token_usage()

token_counter.reset_counts()
print("\nGraph RAG result:")
response_graph_rag = kg_rag_query_engine.query(query)
print(response_graph_rag)
print_token_usage()

token_counter.reset_counts()
print("\nVector RAG result:")
response_vector_rag = vector_rag_query_engine.query(query)
print(response_vector_rag)
print_token_usage()

token_counter.reset_counts()
print("\nGraph + Vector RAG result:")
response_graph_vector_rag = graph_vector_rag_query_engine.query(query)
print(response_graph_vector_rag)
print_token_usage()


Graph RAG result:
LLM Prompt Token Usage: 103
LLM Completion Token Usage: 31
LLM Prompt Token Usage: 154
LLM Completion Token Usage: 139
The treatment for metastatic prostate cancer (M1) and castration-sensitive prostate cancer (CSPC) both involve the use of androgen deprivation therapy (ADT) as a key component. However, the approach and combination with other treatments may differ. In metastatic prostate cancer, ADT is often combined with other therapies such as chemotherapy or novel hormonal agents to enhance effectiveness. In castration-sensitive prostate cancer, ADT is typically the primary treatment, and additional therapies may be considered based on the patient's overall health and disease progression. The goal in both cases is to reduce androgen levels to slow the growth of the cancer, but the specific treatment regimen can vary depending on the stage and characteristics of the cancer.
Prompt Tokens: 257
Completion Tokens: 170
Total Tokens: 427
Estimated Cost: $0.0009

Vector 

In [25]:
# 6. Query with all the Engines

query = "What is the next step after estimating life expectancy for a patient with clinically localized prostate cancer?"

# token_counter.reset_counts()
# print("Text-to-GraphQuery result:")
# response_nl2kg = nl2kg_query_engine.query(query)
# print(response_nl2kg)
# print_token_usage()

token_counter.reset_counts()
print("\nGraph RAG result:")
response_graph_rag = kg_rag_query_engine.query(query)
print(response_graph_rag)
print_token_usage()

token_counter.reset_counts()
print("\nVector RAG result:")
response_vector_rag = vector_rag_query_engine.query(query)
print(response_vector_rag)
print_token_usage()

token_counter.reset_counts()
print("\nGraph + Vector RAG result:")
response_graph_vector_rag = graph_vector_rag_query_engine.query(query)
print(response_graph_vector_rag)
print_token_usage()


Graph RAG result:
LLM Prompt Token Usage: 0
LLM Completion Token Usage: 0


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************Guxy. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}